In [1]:
import os
import sys
import glob
import datetime

import numpy as np
import pandas as pd

In [50]:
gp_table = pd.read_table('crab_gp_kaz_10_2016-2018.csv', sep='\t')
gp_table['Time_series'] = pd.to_datetime(gp_table['Date'], format='%d.%m.%Y', errors='ignore')
gp_table = gp_table.set_index('Time_series')
print(len(gp_table))

715


## Удаление аномальных событий и калибровочных ступений

In [51]:
no_GP = pd.read_table('noGP.csv', sep=',')

In [52]:
no_GP

,дата,номер импульса,Примечание
0,2017-03-02,0,anomal wide
1,2016-06-03,0,anomal wide
2,2016-06-03,1,calib
3,2016-06-03,3,calib
4,2016-06-03,4,calib
5,2016-08-03,0,anomal wide
6,2016-08-03,1,calib
7,2016-08-03,2,calib
8,2016-08-03,3,calib
9,2016-12-06,0,anomal wide


In [53]:
gp_table.head()

,Date,Time start,"Tay, ms","Period, s","Numpointwin, point","Numpulse, a.u.","Median, adc u","STD, adc u",path obs plot,path obs data,"Count of GP, u","point of gp, point","amp of gp, adc u","W50, point","W10, point",path plot
Time_series,,,,,,,,,,,,,,,,
2016-01-15,15.01.2016,22:22:38.4654350,2.4576,6.002102,2439,34,719.50219,11.528548,./obs_plot/15.01.2016_plot_0531+21.png,.\obs_data\15.01.2016_obs_0531+21.csv,1,61785,924.25417,99.0,99.0,./gp_plot/15.01.2016_plot_0531+21_0.png
2016-01-28,28.01.2016,21:31:31.7480869,2.4576,6.002294,2439,34,538.25044,8.978479,./obs_plot/28.01.2016_plot_0531+21.png,.\obs_data\28.01.2016_obs_0531+21.csv,1,27919,705.48172,99.0,99.0,./gp_plot/28.01.2016_plot_0531+21_0.png
2016-02-02,02.02.2016,21:11:52.2432174,2.4576,6.002363,2439,34,801.44774,12.969247,./obs_plot/02.02.2016_plot_0531+21.png,.\obs_data\02.02.2016_obs_0531+21.csv,1,5057,1041.97474,99.0,99.0,./gp_plot/02.02.2016_plot_0531+21_0.png
2016-02-04,04.02.2016,21:04:00.4663017,2.4576,6.002389,2439,34,770.98711,12.384724,./obs_plot/04.02.2016_plot_0531+21.png,.\obs_data\04.02.2016_obs_0531+21.csv,1,39723,912.55581,99.0,99.0,./gp_plot/04.02.2016_plot_0531+21_0.png
2016-02-07,07.02.2016,20:52:12.7584114,2.4576,6.002428,2439,34,734.38958,11.736352,./obs_plot/07.02.2016_plot_0531+21.png,.\obs_data\07.02.2016_obs_0531+21.csv,1,39294,878.88936,99.0,99.0,./gp_plot/07.02.2016_plot_0531+21_0.png


In [54]:
gp_table[(gp_table.index == '2016-06-03') & (gp_table['path plot'].str.contains('_1.png'))]

,Date,Time start,"Tay, ms","Period, s","Numpointwin, point","Numpulse, a.u.","Median, adc u","STD, adc u",path obs plot,path obs data,"Count of GP, u","point of gp, point","amp of gp, adc u","W50, point","W10, point",path plot
Time_series,,,,,,,,,,,,,,,,


In [55]:
for index, row in no_GP.iterrows():
    print(row['дата'], row['номер импульса'])
    gp_table = gp_table.drop(
        gp_table[
            ((gp_table.index == row['дата']) 
             & (gp_table['path plot'].str.contains('_' + str(row['номер импульса']) + '.png')))].index)

2017-03-02 0
2016-06-03 0
2016-06-03 1
2016-06-03 3
2016-06-03 4
2016-08-03 0
2016-08-03 1
2016-08-03 2
2016-08-03 3
2016-12-06 0
2016-12-06 1
2016-11-12 0
2016-06-13 0
2016-06-13 1
2016-06-13 2
2016-06-13 3
2016-06-13 4
2017-07-16 0
2017-07-16 1
2017-07-16 2
2017-07-16 3
2017-07-16 4
2017-07-16 5
2017-07-16 6
2017-07-16 7
2017-07-16 8
2017-07-16 9
2017-07-16 10
2017-07-16 11
2017-07-16 12
2017-07-16 13
2016-06-17 0
2016-07-18 0
2016-07-18 1
2016-07-18 2
2016-09-25 0
2017-09-25 2
2018-04-28 1
2017-05-29 0
2017-05-29 1
2017-05-29 2
2017-05-29 3
2017-05-29 4
2017-05-29 5
2017-05-29 6
2017-05-29 7
2017-05-29 8
2017-05-29 9


In [56]:
len(gp_table)

715

In [79]:
type(gp_table)

pandas.core.frame.DataFrame

In [181]:
gp_table

,Date,Time start,"Tay, ms","Period, s","Numpointwin, point","Numpulse, a.u.","Median, adc u","STD, adc u",path obs plot,path obs data,"Count of GP, u","point of gp, point","amp of gp, adc u","W50, point","W10, point",path plot
Time_series,,,,,,,,,,,,,,,,
2016-01-15,15.01.2016,22:22:38.4654350,2.4576,6.002102,2439,34,719.502190,11.528548,./obs_plot/15.01.2016_plot_0531+21.png,.\obs_data\15.01.2016_obs_0531+21.csv,1,61785,924.25417,99.0,99.0,./gp_plot/15.01.2016_plot_0531+21_0.png
2016-01-28,28.01.2016,21:31:31.7480869,2.4576,6.002294,2439,34,538.250440,8.978479,./obs_plot/28.01.2016_plot_0531+21.png,.\obs_data\28.01.2016_obs_0531+21.csv,1,27919,705.48172,99.0,99.0,./gp_plot/28.01.2016_plot_0531+21_0.png
2016-02-02,02.02.2016,21:11:52.2432174,2.4576,6.002363,2439,34,801.447740,12.969247,./obs_plot/02.02.2016_plot_0531+21.png,.\obs_data\02.02.2016_obs_0531+21.csv,1,5057,1041.97474,99.0,99.0,./gp_plot/02.02.2016_plot_0531+21_0.png
2016-02-04,04.02.2016,21:04:00.4663017,2.4576,6.002389,2439,34,770.987110,12.384724,./obs_plot/04.02.2016_plot_0531+21.png,.\obs_data\04.02.2016_obs_0531+21.csv,1,39723,912.55581,99.0,99.0,./gp_plot/04.02.2016_plot_0531+21_0.png
2016-02-07,07.02.2016,20:52:12.7584114,2.4576,6.002428,2439,34,734.389580,11.736352,./obs_plot/07.02.2016_plot_0531+21.png,.\obs_data\07.02.2016_obs_0531+21.csv,1,39294,878.88936,99.0,99.0,./gp_plot/07.02.2016_plot_0531+21_0.png
2016-02-10,10.02.2016,20:40:25.0676345,2.4576,6.002465,2439,34,530.659450,8.431679,./obs_plot/10.02.2016_plot_0531+21.png,.\obs_data\10.02.2016_obs_0531+21.csv,1,37770,642.85162,99.0,99.0,./gp_plot/10.02.2016_plot_0531+21_0.png
2016-02-10,10.02.2016,20:40:25.0676345,2.4576,6.002465,2439,34,530.659450,8.431679,./obs_plot/10.02.2016_plot_0531+21.png,.\obs_data\10.02.2016_obs_0531+21.csv,1,44730,625.72359,99.0,99.0,./gp_plot/10.02.2016_plot_0531+21_1.png
2016-02-13,13.02.2016,20:28:37.3627548,2.4576,6.002501,2439,34,737.515830,11.689590,./obs_plot/13.02.2016_plot_0531+21.png,.\obs_data\13.02.2016_obs_0531+21.csv,1,8564,971.96368,99.0,99.0,./gp_plot/13.02.2016_plot_0531+21_0.png
2016-02-18,18.02.2016,20:08:57.8583717,2.4576,6.002557,2439,34,759.471920,12.946473,./obs_plot/18.02.2016_plot_0531+21.png,.\obs_data\18.02.2016_obs_0531+21.csv,1,42454,1138.63782,99.0,99.0,./gp_plot/18.02.2016_plot_0531+21_0.png


In [49]:
gp_table.to_csv('crab_gp_kaz_10_2016-2018.csv',  sep='\t', header=True, index=False)

In [83]:
row['номер импульса']

0

In [98]:
gp_table.drop(
    gp_table[
        (gp_table.index == '2017-02-03')
        & (gp_table['path plot'].str.contains('_1.png'))].index)

TypeError: 'NoneType' object is not iterable

In [35]:
df = pd.DataFrame(np.arange(24).reshape(6,4),columns=['A', 'B', 'C', 'D'])

In [40]:
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,ada
2,8,9,10,bada
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [37]:
dd = pd.DataFrame([[1, 'ada'],[2, 'bada'],[5, '3']],columns=['A1', 'B1'])

In [32]:
dd

,A1,B1
0,1,ada
1,2,bada
2,5,3


In [41]:
for index, row in dd.iterrows():
    df = df.drop(df[(df.index == row['A1']) & (df['D'].str.contains(row['B1']))].index)

In [42]:
df

,A,B,C,D
0,0,1,2,3
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
